In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

df_train = pd.read_csv('/content/gdrive/My Drive/DACON/train.csv')
df_test = pd.read_csv('/content/gdrive/My Drive/DACON/test.csv')
sub = pd.read_csv('/content/gdrive/MyDrive/DACON/sample_submission.csv')
df_info = pd.read_csv('/content/gdrive/My Drive/DACON/snp_info.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df_info

,SNP_id,name,chrom,cm,pos
0,SNP_01,BTA-19852-no-rs,2,67.05460,42986890
1,SNP_02,ARS-USMARC-Parent-DQ647190-rs29013632,6,31.15670,13897068
2,SNP_03,ARS-BFGL-NGS-117009,6,68.28920,44649549
3,SNP_04,ARS-BFGL-NGS-60567,6,77.87490,53826064
4,SNP_05,BovineHD0600017032,6,80.50150,61779512
5,SNP_06,BovineHD0600017424,6,80.59540,63048481
6,SNP_07,Hapmap49442-BTA-111073,6,80.78000,64037334
7,SNP_08,BovineHD0600018638,6,82.68560,67510588
8,SNP_09,ARS-BFGL-NGS-37727,6,86.87400,73092782
9,SNP_10,BTB-01558306,7,62.06920,40827112


전처리를 해보자

In [4]:
df_test.head()

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,TEST_000,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,TEST_001,0,0,0,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,TEST_002,0,0,0,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,TEST_003,0,0,0,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,TEST_004,0,0,0,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G


In [5]:
#레이블 인코딩을 위해 유니크한 값을 체크해보자
df_list = [df_train, df_test]
snp_list = ["SNP_01", "SNP_02", "SNP_03", "SNP_04", "SNP_05", "SNP_06", "SNP_07",
            "SNP_08", "SNP_09", "SNP_10", "SNP_11", "SNP_12", "SNP_13", "SNP_14", "SNP_15"]
lst = []

for df in df_list:
  for snp in snp_list:
    li = df[snp].unique()
    for l in li:
      lst.append(l)

set(lst)

{'A A', 'A G', 'C A', 'C C', 'G A', 'G G'}

Label Encoding


In [6]:
class_map ={'A':0,
            'B':1,
            'C':2,
            }
df_train['class'] = df_train['class'].map(class_map)
df_train.head()

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,1
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,2
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,1
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,0
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,2


In [7]:
 
snp_list = ["SNP_01", "SNP_02", "SNP_03", "SNP_04", "SNP_05", "SNP_06", "SNP_07",
            "SNP_08", "SNP_09", "SNP_10", "SNP_11", "SNP_12", "SNP_13", "SNP_14", "SNP_15"]

label_map = {'A A': 0 , 'A G': 1, 'C A': 2, 'C C': 3, 'G A': 4, 'G G': 5}

for snp in snp_list:
    df_train[snp] = df_train[snp].map(label_map)

for snp in snp_list:
    df_test[snp] = df_test[snp].map(label_map)

df_train.head()


,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,5,1,0,4,2,...,0,5,0,5,1,0,0,0,0,1
1,TRAIN_001,0,0,0,2,1,1,2,0,0,...,0,4,0,1,0,4,5,0,0,2
2,TRAIN_002,0,0,0,2,5,5,0,4,3,...,0,4,4,1,0,0,0,0,0,1
3,TRAIN_003,0,0,0,1,0,5,0,4,0,...,5,0,5,1,5,5,5,0,5,0
4,TRAIN_004,0,0,0,2,5,5,3,0,3,...,0,0,0,5,0,0,1,0,4,2


In [8]:
df_train = df_train.drop(columns=["id","father","mother","gender"])
df_test = df_test.drop(columns=["id","father","mother","gender"])

In [9]:
df_train.head()

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,2,5,1,0,4,2,0,0,5,0,5,1,0,0,0,0,1
1,2,1,1,2,0,0,1,0,4,0,1,0,4,5,0,0,2
2,2,5,5,0,4,3,5,0,4,4,1,0,0,0,0,0,1
3,1,0,5,0,4,0,5,5,0,5,1,5,5,5,0,5,0
4,2,5,5,3,0,3,0,0,0,0,5,0,0,1,0,4,2


In [10]:
df_test.head()

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,1,1,5,0,4,0,1,5,4,4,1,1,4,5,2,4
1,2,5,1,3,5,3,0,0,0,0,5,1,0,0,0,0
2,2,5,1,0,0,2,1,0,0,0,1,0,4,5,0,5
3,2,5,1,2,0,3,0,0,0,0,5,0,4,1,0,0
4,1,0,5,0,5,0,5,5,0,5,1,5,4,5,0,5


모델링을 해보자 

: Label Encoding은 값의 크기에 따라 가중치로 인식하여 분류가 제대로 되지 않을 수 있으므로 일반 ML에서는 권장하지 않는다. 
하지만 Tree계열이라면 상관이 없으므로 Tree계열의 모델링을 사용한다 
-> Decision Tree/Random Forest

In [11]:
#훈련 및 테스트 데이터 변수 준비
X_train = df_train.drop(columns=["class"])
y_train = df_train["class"]

X_test = df_test

In [14]:
r = 0
classifiers = {"kNN":KNeighborsClassifier(), 
               "SVC":SVC(),
               "DTC":DecisionTreeClassifier(random_state=r) ,
               "RFC":RandomForestClassifier(random_state=r) ,
               "GNB":GaussianNB(),
               "MLP":MLPClassifier(random_state=r),
               "XGB":XGBClassifier(random_state=r),
               "LGB": LGBMClassifier(random_state=r)}

for name in classifiers: 
    score = cross_val_score(classifiers[name], X_train, y_train, cv=5).mean()
    print(name, score.round(4))    

kNN 0.9049
SVC 0.9466
DTC 0.9199
RFC 0.9505
GNB 0.9124


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:

MLP 0.9391
XGB 0.939
LGB 0.9391


In [15]:
clf_list = [('clf1', XGBClassifier(random_state=r)),
            ('clf2', RandomForestClassifier(random_state=r)),
            ('clf3', LGBMClassifier(random_state=r)),
            ('clf4', DecisionTreeClassifier(random_state=r))]

In [16]:
clf = RandomForestClassifier(random_state=r)
accuracy = cross_val_score(clf, X_train, y_train)

accuracy.mean()

0.9505079825834543

In [17]:
final_model = clf 
final_model.fit(X_train, y_train)

print(final_model)

RandomForestClassifier(random_state=0)


In [18]:
X_test = df_test

y_pred = final_model.predict(X_test)

print(y_pred)

[0 1 2 1 0 1 2 1 0 0 2 1 1 0 1 1 0 1 1 2 1 1 1 0 1 1 1 0 0 1 0 0 1 2 0 1 2
 1 1 2 0 1 2 1 1 1 1 2 1 2 0 1 0 1 1 1 2 0 1 2 0 1 2 2 2 0 1 0 0 1 1 1 0 0
 2 1 2 1 1 1 2 1 0 1 1 1 1 1 2 0 1 1 2 1 1 2 0 1 0 2 0 1 1 2 0 0 2 1 0 1 2
 1 1 1 1 0 0 2 1 1 0 1 1 2 2 1 2 1 0 1 0 0 1 1 1 2 0 0 1 0 0 0 2 1 1 1 0 1
 2 0 0 1 0 1 1 0 0 1 2 0 1 2 2 1 0 0 2 1 1 0 1 2 2 1 1]


In [19]:
y_pred = np.where(y_pred == 0, "A", y_pred)
y_pred = np.where(y_pred == "1", "B", y_pred)
y_pred = np.where(y_pred == "2", "C", y_pred)

In [20]:
y_pred

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'B', 'A', 'B', 'B', 'C', 'C', 'B', 'C', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype='<U21')

In [21]:
sub["class"] = y_pred # 이전 셀에서 최종 모델로 생성한 답안

sub.to_csv(f"snp_sydney.csv", index=False) # 생성된 csv 파일을 좌측 폴더에서 다운로드하여 제출    